# Курсовая. Эмпирическая часть

### Здесь приведены все операции, который я выполнял, чтобы получить результаты, приведённые в документе. Я также применял анализ отзывов по темам с помощью LDA, но он не дал осмысленных результатов.

Импортируем необходимые библиотеки

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import random
from scipy import stats
from statsmodels.stats import proportion

In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import warnings 
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_excel('finalAll.xlsx')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9624 entries, 0 to 9623
Data columns (total 21 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   отзыв                                                                         9624 non-null   object 
 1   Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный)       9623 non-null   object 
 2   ответ психолога на отзыв                                                      5229 non-null   object 
 3   ссылка на профиль
                                                            9624 non-null   object 
 4   пол (м или ж)                                                                 9624 non-null   object 
 5   возраст                                                                       8355 non-null   float64
 6   опыт работы психологом (количест

Преобразуем в числа стоимость консультации

In [7]:
df[df.columns[17]] = pd.to_numeric(df[df.columns[17]])

Найдём связь между ценой и долей негативных отзывов

In [8]:
df['is_negative'] = df[df.columns[1]].apply(lambda x: 1 if x == 'о' else 0)

In [9]:
id_negative_age = df.groupby(df.columns[3]).mean()[[df.columns[17], 'is_negative']].dropna()

In [10]:
res = stats.pearsonr(id_negative_age.iloc[:, 0], id_negative_age.iloc[:, 1])
print(f'Correlation between price and proportion of negative reviews\nr Pearson = {res[0]:.3f}, p-value = {res[1]:.3f}')

Correlation between price and proportion of negative reviews
r Pearson = 0.125, p-value = 0.033


Но не было найдено аналогичной связи между долей негативных отзывов и (возрастом, опытом работы)

Распределение психологов по полу в абсолютных значениях и в процентах

In [11]:
sex_psychologists = df.groupby([df.columns[3], df.columns[4]]).mean().reset_index()[df.columns[4]]
sex_psychologists.value_counts()

ж    214
м     86
Name: пол (м или ж), dtype: int64

In [12]:
sex_psychologists.value_counts(normalize=True).round(2)

ж    0.71
м    0.29
Name: пол (м или ж), dtype: float64

Ниже процентное и абсолютное соотношение негативных отзывов, где 1 - негативные отзывы.

Психологи мужского пола

In [13]:
male_psychologists = df[df[df.columns[4]] == 'м']['is_negative']
male_psychologists.value_counts(normalize=True).round(3)

0    0.977
1    0.023
Name: is_negative, dtype: float64

In [14]:
male_psychologists.value_counts()

0    3000
1      70
Name: is_negative, dtype: int64

Психологи женского пола

In [15]:
female_psychologists = df[df[df.columns[4]] == 'ж']['is_negative']
female_psychologists.value_counts(normalize=True).round(3)

0    0.982
1    0.018
Name: is_negative, dtype: float64

In [16]:
female_psychologists.value_counts()

0    6438
1     116
Name: is_negative, dtype: int64

Видим разницу в соотношении отзывов, но она небольшая (.023 vs .018), поэтому даже не стоит рассматривать статистические тесты.

In [17]:
# proportion.proportions_ztest(np.array([3000, 6438]), np.array([3000 + 70, 6438 + 116]), alternative='two-sided')

Соотношение отзывов без деления по полу психолога

In [18]:
(df[df.columns[1]] == 'о').value_counts()

False    9438
True      186
Name: Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный), dtype: int64

In [19]:
df[df.columns[1]].value_counts()

п    8878
н     559
о     186
Name: Разметка отзыва (п - положительный, н - нейтральный, о - отрицательный), dtype: int64

На ~51 положительный или нейтральный отзыв приходится один отрицательный

In [20]:
(8878 + 559) / 186

50.736559139784944

Функция процессинга текста имеет такой вид, так как под windows лемматизация при помощи pymystem3 выполняется неадекватно долго и необходимы обходные пути, чтобы ускорить функцию, оставив прежний функционал.

In [21]:
def preprocess_text(texts):
    russian_stopwords = stopwords.words("russian")
    mystem = Mystem() 
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(texts, 1000)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' br ' for txt in txtp])

        words = mystem.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != ''\
            and txt not in russian_stopwords and all(i not in punctuation for i in txt) and len(txt.strip()) > 1 and not any(char.isdigit() for char in txt):
                if txt == 'br':
                    res.append(doc)
                    doc = []
                else:
                    doc.append(txt)
    return res

In [22]:
%%time
reviews_processed = preprocess_text(df['отзыв'])

Wall time: 36.3 s


In [23]:
def get_freq_words(reviews_processed, sentiment, condition=None):
    """
    Функция принимает таблицу с почищенными отзывами, тип отзыва и опционально дополнительное условие.
    Выводит отсортированные частоты слов в формате pd.Series"""
    list_words = []
    if condition is not None:
        [list_words.extend(i) for i in pd.Series(reviews_processed)[(df[df.columns[1]] == sentiment) & condition]]
    else:
        [list_words.extend(i) for i in pd.Series(reviews_processed)[df[df.columns[1]] == sentiment]]
    return pd.Series(list_words).value_counts(normalize=True)

Ниже нахожу частоты слов в отрицательных и положительных отзывах

In [24]:
pos = get_freq_words(reviews_processed, 'п')
neg = get_freq_words(reviews_processed, 'о')

Нахожу частоты положительных отзывов у психологов в зависимости от пола

In [25]:
pos_male = get_freq_words(reviews_processed, 'п', df[df.columns[4]] == 'м')
pos_female = get_freq_words(reviews_processed, 'п', df[df.columns[4]] == 'ж')

Ниже цикл находит слова, которые встречаются у психологов обоих полов. И записывает у этого слова соотношение частот (== шансы).

In [26]:
def get_odds(first_freq, second_freq):
    """Получаем шансы слова оказаться в первом словаре. Выводим отсортированные значения в формате pd.Series"""
    dict_odds = {}
    for i in first_freq.index:
        if i in second_freq.index:
            dict_odds[i] = first_freq[i] / second_freq[i]
    return pd.Series(dict_odds).sort_values()

Находим шансы, записываем в словарь и выводим в текстовый файл

In [27]:
def return_limited_odds(odds_dict, second_dict, min_in_second=5, min_multiplicator=2, reverse=False):
    """Главная цель функции - перевернуть шансы и ограничить длинный список некоторыми параметрами, а именно:
    минимальным значением шанса и минимальным кол-вом во втором словаре.
    При reverse==False будут первыми выходить шансы второго словаря"""
    dict_out = {}
#     Можем перевернуть порядок и все значения, чтобы шансы лучше воспринимались на глаз
    if reverse == True:
        odds_dict = odds_dict.iloc[::-1]
        odds_dict = 1 / odds_dict
    for i in odds_dict.index:
        odds = 1 / odds_dict[i]
        if odds < min_multiplicator:
#         прерываем цикл, если встретилось слово с шансами, ниже указанного
            break
        if second_dict[i] / second_dict.min() >= min_in_second:
            dict_out[i] = (odds).round(2)
    return dict_out

In [28]:
def save_dict_txt(filename, odds_dict):
    """Сохраняем полученный словарик в чистый текстовый файл. Слева слово, справа - шанс"""
    with open(filename, 'w') as f:
        for k, v in odds_dict.items():
            f.write(str(k) + ' '+ str(v) + '\n')

In [29]:
multi_freq_sex = get_odds(pos_female, pos_male)

In [30]:
odds_female_pos = return_limited_odds(multi_freq_sex, pos_male, min_in_second=5, min_multiplicator=1.5, reverse=True)
odds_male_pos = return_limited_odds(multi_freq_sex, pos_male, min_in_second=5, min_multiplicator=2, reverse=False)

In [31]:
save_dict_txt('txt_out/female_pos_psychologists.txt', odds_female_pos)
save_dict_txt('txt_out/male_pos_psychologists.txt', odds_male_pos)

Далее операции аналогичные, но находим шансы слов уже между отрицательными и положительными отзывами

In [32]:
odds_neg_pos = get_odds(pos, neg)

In [33]:
odds_pos = return_limited_odds(odds_neg_pos, neg, min_in_second=5, min_multiplicator=1.3, reverse=True)
odds_neg = return_limited_odds(odds_neg_pos, neg, min_in_second=2, min_multiplicator=2, reverse=False)

In [34]:
save_dict_txt('txt_out/pos_psychologists.txt', odds_pos)
save_dict_txt('txt_out/neg_psychologists.txt', odds_neg)

Не очень удобно, но оригинальные отзывы можно прямо здесь почитать.

In [35]:
rev_neg = pd.Series(reviews_processed)[df[df.columns[1]] == 'о']
rev_pos = pd.Series(reviews_processed)[df[df.columns[1]] == 'п']

In [36]:
rev_pos[rev_pos.apply(lambda x: 'черепаха' in x)]

17      [здравствовать, оксана, впечатление, работа, о...
9103    [древность, человек, верить, земля, держаться,...
dtype: object

In [37]:
ix = rev_pos[rev_pos.apply(lambda x: 'черепаха' in x)].index
with pd.option_context('display.max_colwidth', None):
    print(df.iloc[ix, 0])

17      Здравствуйте, Оксана!Впечатления от работы с Вами у меня очень хорошие. С Вами легко и весело, а это для любого серьезного дела несомненный плюс.Самое ценное, что я вынесла из консультации - ясное осознание своих эмоций (которые Вы так четко озвучили), суть моего сопротивления, и что нужно все-таки принимать себя, без этого никак. Что нужно искать в себе цветные краски, а не зацикливаться только на черном. Сейчас вот именно это и практикую. Позволила себе выспаться до отвала, и не осуждаю себя за это. "Черт возьми, я же в отпуске! Могу же себе позволить!" И могу позволить заниматься тем, что доставляет удовольствие, а не уборкой, в которой большой необходимости-то и нет на самом деле. Ищу в себе цветное, учусь смотреть на себя с разных сторон. Да, я лентяйка и тормоз, но зато добрый человек, ответственный скрупулезный работник, усидчивая и терпеливая, нежная и ласковая... И правда, почему это я только на черном в себе зацикливаюсь?.. Видимо, природе зачем-то нужны черепахи, коа

Также можно проанализировать N-граммы, но они менее информативны, чем шансы слов. Возможно, было бы лучше так же найти шансы N-граммов, но результаты будут менее надёжны, так как уменьшатся частоты элементов (в данном случае - биграммов).

In [38]:
def count_ngrams(series: pd.Series, n: int) -> pd.Series:
    ngrams = series.copy().str.split(' ').explode()
    for i in range(1, n):
        ngrams += ' ' + ngrams.groupby(level=0).shift(-i)
        ngrams = ngrams.dropna()
    return ngrams.value_counts()    

In [39]:
df['review_lem'] = [' '.join(i) for i in reviews_processed]

In [40]:
print(count_ngrams(df.loc[df[df.columns[1]] == 'о', 'review_lem'], 2).index[:50])

Index(['данный специалист', 'демо консультация', 'время консультация',
       'очный консультация', 'закрывать тема', 'консультация данный',
       'весь время', 'задавать вопрос', 'обращаться психолог',
       'проблема решать', 'взять тема', 'свой проблема', 'решать проблема',
       'татьяна михайловна', 'сказать это', 'это нормально',
       'становиться плохо', 'складываться впечатление', 'работа психолог',
       'консультация скайп', 'год назад', 'решение проблема', 'очень жаль',
       'консультация это', 'сей пора', 'тема закрывать', 'обращаться помощь',
       'демо тема', 'опускать рука', 'весь это', 'дальнейший работа',
       'специалист который', 'оставаться неприятный', 'свой вопрос',
       'https www', 'несколько день', 'сделать это', 'психолог который',
       'это делать', 'консультация специалист', 'психолог мочь',
       'консультация сайт', 'получать помощь', 'ставить диагноз',
       'некоторый время', 'свой время', 'неприятный впечатление', 'весь таки',
       '

В заключении я рисовал облака слов. Но, если честно, на них непросто найти различия между положительными и отрицательными отзывами. Эти графики не приносят новой информации в удобной форме. Хоть и выглядят внушительно, но пользы от них немного.